In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
from datetime import datetime

#https://towardsdatascience.com/building-a-real-time-dashboard-using-python-plotly-library-and-web-service-145f50d204f0


In [2]:
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])
df.head()

,attributes,geometry
0,"{'OBJECTID': 1, 'Province_State': None, 'Count...","{'x': 67.70995300000004, 'y': 33.93911000000003}"
1,"{'OBJECTID': 2, 'Province_State': None, 'Count...","{'x': 20.168300000000045, 'y': 41.15330000000006}"
2,"{'OBJECTID': 3, 'Province_State': None, 'Count...","{'x': 1.6596000000000686, 'y': 28.033900000000..."
3,"{'OBJECTID': 4, 'Province_State': None, 'Count...","{'x': 1.5218000000000416, 'y': 42.50630000000007}"
4,"{'OBJECTID': 5, 'Province_State': None, 'Count...","{'x': 17.87390000000005, 'y': -11.202699999999..."


In [3]:
df["attributes"][0]

{'OBJECTID': 1,
 'Province_State': None,
 'Country_Region': 'Afghanistan',
 'Last_Update': 1597933680000,
 'Lat': 33.93911,
 'Long_': 67.709953,
 'Confirmed': 37856,
 'Recovered': 27681,
 'Deaths': 1385,
 'Active': 8790,
 'Admin2': None,
 'FIPS': None,
 'Combined_Key': 'Afghanistan'}

In [4]:
data_list = df["attributes"].tolist()
df_final = pd.DataFrame(data_list)
df_final.set_index("OBJECTID")
df_final = df_final[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Last_Update"]]

In [5]:
df_final.head()

,Country_Region,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Last_Update
0,Afghanistan,None,33.93911,67.709953,37856,1385,27681,1.597934e+12
1,Albania,None,41.15330,20.168300,7812,234,3928,1.597934e+12
2,Algeria,None,28.03390,1.659600,39847,1402,27971,1.597934e+12
3,Andorra,None,42.50630,1.521800,1024,53,875,1.597934e+12
4,Angola,None,-11.20270,17.873900,2015,92,698,1.597934e+12


In [6]:
def convertTime(t):
    t = int(t)
    return datetime.fromtimestamp(t)

df_final = df_final.dropna(subset=["Last_Update"])
df_final["Province_State"].fillna(value="", inplace=True)

df_final["Last_Update"]= df_final["Last_Update"]/1000
df_final["Last_Update"] = df_final["Last_Update"].apply(convertTime)

df_final.head()

,Country_Region,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Last_Update
0,Afghanistan,,33.93911,67.709953,37856,1385,27681,2020-08-20 19:58:00
1,Albania,,41.15330,20.168300,7812,234,3928,2020-08-20 19:58:00
2,Algeria,,28.03390,1.659600,39847,1402,27971,2020-08-20 19:58:00
3,Andorra,,42.50630,1.521800,1024,53,875,2020-08-20 19:58:00
4,Angola,,-11.20270,17.873900,2015,92,698,2020-08-20 19:58:00


In [7]:
df_total = df_final.groupby("Country_Region", as_index=False).agg(
    {
        "Confirmed" : "sum",
        "Deaths" : "sum",
        "Recovered" : "sum"
    }
)

df_total.head()

,Country_Region,Confirmed,Deaths,Recovered
0,Afghanistan,37856,1385,27681
1,Albania,7812,234,3928
2,Algeria,39847,1402,27971
3,Andorra,1024,53,875
4,Angola,2015,92,698


In [8]:
total_confirmed = df_final["Confirmed"].sum()
total_recovered = df_final["Recovered"].sum()
total_deaths = df_final["Deaths"].sum()

In [9]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

In [10]:
fig = make_subplots(
    rows = 4, cols = 6,

    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)


message = df_final["Country_Region"] + " " + df_final["Province_State"] + "<br>"
message += "Confirmed: " + df_final["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df_final["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df_final["Recovered"].astype(str) + "<br>"
message += "Last updated: " + df_final["Last_Update"].astype(str)
df_final["text"] = message

fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = df_final["Long_"],
        lat = df_final["Lat"],
        hovertext = df_final["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = df_final['Confirmed'],
            cmax = df_final['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)


fig.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(df_final["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
    geo = dict(
            projection_type="orthographic",
            showcoastlines=True,
            landcolor="white", 
            showland= True,
            showocean = True,
            lakecolor="LightBlue"
    ),

    annotations=[
        dict(
            text="Source: https://bit.ly/3aEzxjK",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.35,
            y=0)
    ]
)

fig.write_html('first_figure.html', auto_open=True)